# Dimensionality Reduction

It is useful to reduce the dimensionality of our data in various situations such as we do not have enough points for the size of the data, we think that our data comes from a low-dimensional space or we want to interpret/visualize the data. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()

Let's go over the `digits` dataset once more.

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()
X = digits["data"]
y = digits["target"]

fig, axes = plt.subplots(10, 10, figsize = (8, 8),
                         subplot_kw = {"xticks": [], "yticks": []},
                         gridspec_kw = dict(hspace = 0.1, wspace = 0.1))

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap = "binary", interpolation = "nearest")
    ax.text(0.05, 0.05, str(digits.target[i]),
            transform = ax.transAxes, color = "green")

Our data is between 0 and 16, let's scale it between 0 and 1.

In [ ]:
X = X / 16.0

## Principal Component Analysis (PCA)

Let's start with PCA, which is a multi-purpose and fundamental algorithm. It is perhaps the most commonly used dimensionality reduction approach. PCA separates data into uncorrelated (0 covariance) components. It is readily available in sckit-learn. It has the `fit`, `transform` and `fit_transform` functions that we are familiar with.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 64)
pca.fit(X)
print(pca)

In [ ]:
plt.figure(figsize = (12,8))
explained_variance_cs = np.cumsum(pca.explained_variance_ratio_)
plt.plot(explained_variance_cs)
plt.xlabel("Number of components")
plt.ylabel("Explained Variance")
plt.show()

In [ ]:
print('Number of components for at least 90% variance explanation:', 
      np.where(explained_variance_cs>0.90)[0][0])

In [ ]:
print(pca.components_.shape)

Let's visualize the components

In [ ]:
fig, axes = plt.subplots(8, 8, figsize = (8, 8),
                         subplot_kw = {"xticks": [], "yticks": []},
                         gridspec_kw = dict(hspace = 0.1, wspace = 0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(8, 8), cmap = "binary", interpolation = "nearest")

In [ ]:
pca40 = PCA(n_components = 40)
X_transformed = pca40.inverse_transform(pca40.fit_transform(X))

fig, axes = plt.subplots(10, 10, figsize = (8, 8),
                         subplot_kw = {"xticks": [], "yticks": []},
                         gridspec_kw = dict(hspace = 0.1, wspace = 0.1))

for i, ax in enumerate(axes.flat):
    ax.imshow(X_transformed[i,:].reshape(8, 8), cmap = "binary", interpolation = "nearest")
    ax.text(0.05, 0.05, str(digits.target[i]),
            transform = ax.transAxes, color = "green")

Let's project our multi-dimensional data (64 for the digits dataset) to 2 dimensions and visualize the points

In [ ]:
pca2 = PCA(n_components = 2)
Xt = pca2.fit_transform(X)
print(pca2)

print(Xt.shape)

print(np.sum(pca2.explained_variance_ratio_))

In [ ]:
def plotDigitProjections(Xin, y, title = None):
    plt.figure(figsize = (16,10))
    for c in range(10):
        indexes = np.where(y == c)[0]
        x = Xin[indexes]
        plt.scatter(x[:, 0], x[:, 1], label = c)
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.legend()
    if title:
        plt.title(title)
    plt.show()

In [ ]:
plotDigitProjections(Xt, y, "PCA")

## Linear Discriminant Analysis (LDA)

PCA is an unsupervised method. It does not take the labels into accoutt. LDA uses label information for dimensionality reduction.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components = 2)
Xtlda = lda.fit_transform(X, y)
print(np.sum(lda.explained_variance_ratio_))

In [ ]:
plotDigitProjections(Xtlda, y, "LDA")

The explained variance for the LDA method is higher than the PCA method!

## Using PCA for Data Whitening

Some problems require each principal component to be on the same scale. Converting data to its principal components and scaling each component to have unit variance is called data whitening. Data whitening ensures that all dimensions have the same scale relative to each other. This can be good (naturally scaled data) or bad (increasing importance of noisy components). In addition, some methods assume unit covariance (e.g., LDA assumes that every class has the same covariance matrix)onto separating the principal components of the data.

It's easy to use the `PCA` class in `scikit-learn` for data bleaching:

In [ ]:
pca = PCA(whiten = True)
Xwhitened = pca.fit_transform(X)

fig, axes = plt.subplots(1, 2, figsize = (12, 8))
axes[0].matshow(np.cov(X.T))
axes[0].set_title("Covariance of the Original Data", {"fontsize": 18})
axes[1].matshow(np.cov(Xwhitened.T))
axes[1].set_title("Covariance of the Whitened Data", {"fontsize": 18})
plt.show()

## Nonlinear Methods

PCA ve LDA are linear methods. However, the data might have a non-linear subspace or manifold. We need nonlinear methods for this type of data.

We will only demonstrate the methods and will not go into detail.

### Multi-Dimensional Scaling

In [ ]:
from sklearn.manifold import MDS

#Takes a while to run
mds = MDS(n_components = 2)
Xtmds = mds.fit_transform(X)
plotDigitProjections(Xtmds, y, "MDS")

### Isomap

In [ ]:
from sklearn.manifold import Isomap
Xtiso = Isomap(n_components = 2).fit_transform(X)
plotDigitProjections(Xtiso, y, "Isomap")

### t-SNE

This is one of the most commonly used method for data visualization by mapping it to 2D or 3D.

However, it cannot perform out-of-sample dimensionality reduction (hence only used for visualizing data). As a result, you should not use it for learning (there are exceptions to this rule, e.g., for feature extraction from a known and finite set of data)

There is a useful [FAQ](https://lvdmaaten.github.io/tsne/) from the algorithm developer.

See also https://projector.tensorflow.org/ for a variety of methods for your visualization needs.

In [ ]:
from sklearn.manifold import TSNE
Xtsne = TSNE(n_components = 2).fit_transform(X)
plotDigitProjections(Xtsne, y, "t-SNE")

In [ ]:
def plotDigitProjections3D(Xin, y, title = None):
    ax = plt.axes(projection='3d')
    for c in range(10):
        indexes = np.where(y == c)[0]
        x = Xin[indexes]
        ax.scatter(x[:, 0], x[:, 1], x[:, 2], label = c)
    ax.set_xlabel('Component 1')
    ax.set_ylabel('Component 2')
    ax.set_zlabel('Component 3')

    plt.legend()
    if title:
        plt.title(title)
    plt.show()

In [ ]:
#3D version
%matplotlib notebook 

Xtsne3D = TSNE(n_components = 3).fit_transform(X)

plotDigitProjections3D(Xtsne3D,y)

In [ ]:
%matplotlib inline 

**Exercise:** Use the digits data-set along with a dimensionality reduction method and perform classification using Pipelines. The baseline is provided for you.

In [ ]:
print(X.shape, np.max(X), np.min(X), X.dtype, y.shape)
print(np.unique(y))
sns.countplot(x = y)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify = y, test_size=0.4, shuffle = True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logRegClassifier = LogisticRegression(C = 1.0, max_iter = 10000)

logRegClassifier.fit(Xtrain, ytrain)

In [ ]:
ytrainPred = logRegClassifier.predict(Xtrain)
print("Training set performance:", accuracy_score(ytrain, ytrainPred))
ypred = logRegClassifier.predict(Xtest)
print("Test set performance:", accuracy_score(ytest, ypred))

In [ ]:
# Pipeline: dimensionality reduction + classifier


**Home Exercise:** Try it with other classifiers.

Another example (Eigenfaces):

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person = 60)
print(faces.target_names)
print(faces.images.shape)

from sklearn.decomposition import PCA
pca = PCA(150, svd_solver = "randomized")
pca.fit(faces.data)


fig, axes = plt.subplots(3, 8, figsize = (9, 4),
                         subplot_kw = {"xticks": [], "yticks": []},
                         gridspec_kw = dict(hspace = 0.1, wspace = 0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(62, 47), cmap = "bone")

components = pca.transform(faces.data)
projected = pca.inverse_transform(components)

fig, ax = plt.subplots(2, 10, figsize = (10, 2.5),
                       subplot_kw = {"xticks": [], "yticks": []},
                       gridspec_kw = dict(hspace = 0.1, wspace = 0.1))
for i in range(10):
    ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap = "binary_r")
    ax[1, i].imshow(projected[i].reshape(62, 47), cmap = "binary_r")

ax[0, 0].set_ylabel("full-dim\ninput")
ax[1, 0].set_ylabel("150-dim\nreconstruction")
plt.show()

In [ ]:
import umap

u = umap.UMAP(n_components=2)
Xumap = u.fit_transform(X)

plotDigitProjections(Xumap, y)


In [ ]:
plotDigitProjections(Xtsne, y)

In [ ]:
u.transform